In [1]:
import numpy as np
import pandas as pd
import ast

import h3
import folium
from folium import Map, Marker, GeoJson
import branca.colormap as cm
import branca

from geojson.feature import Feature, FeatureCollection
import json

In [2]:
hex_resolution = 8
size_school = 1000

# prepare hexagons for quadrants and schools

In [3]:
adm_data = pd.read_csv("data/adm_data_child_share.csv")
matching = pd.read_excel("data/Справочник_соотношения_секторов_500х500м_и_адм_районов_МСК.xlsx")
matching = matching.merge(adm_data[["adm_zid", "school_year"]])

quadrants = pd.read_csv("data/df_500m_coords.csv")

quadrants = quadrants.merge(matching.drop_duplicates("cell_zid")[["cell_zid", "adm_zid", "school_year"]], left_on="zid", right_on="cell_zid").drop(columns="cell_zid")
quadrants["coords"] = quadrants.coords.apply(ast.literal_eval)

def wrapper(coords):
    return {
        "coordinates": [coords],
        "type": "Polygon",
    }

quadrants["pol_coords"] = quadrants["coords"].apply(wrapper)

quadrants["hexagons"] = quadrants["pol_coords"].apply(h3.polyfill_geojson, res=hex_resolution)

quadrants["population_per_hex"] = quadrants.customers_cnt_home / quadrants.hexagons.apply(len)
quadrants["opacity"] = np.log(quadrants.population_per_hex + 1)/ np.log(quadrants.population_per_hex.max())

In [4]:
quadrants["pupils_per_hex"] = quadrants["population_per_hex"] * quadrants["school_year"]

In [5]:
with open("data/isohrones.npy", "rb") as f:
    polygons_list = np.load(f, allow_pickle=True)
    
for i in range(len(polygons_list)):
    polygons_list[i]['hexagons'] = h3.polyfill_geojson(polygons_list[i]['geometry'], hex_resolution)
    
df_hex = pd.DataFrame()

for i in range(len(polygons_list)):
    df_temp = pd.DataFrame(polygons_list[i]['hexagons'], columns=['hexagons'])
    df_temp['group_index'] = polygons_list[i]['properties']['group_index']
    df_temp['latitude'] = polygons_list[i]['properties']['center'][0]
    df_temp['longitude'] = polygons_list[i]['properties']['center'][1]
    df_temp['area'] = polygons_list[i]['properties']['area']
    df_temp['reachfactor'] = polygons_list[i]['properties']['reachfactor']
    df_temp['total_pop'] = polygons_list[i]['properties']['total_pop']
    df_hex = pd.concat([df_hex, df_temp])

In [6]:
schools = [polygon["hexagons"] for polygon in polygons_list]

# calculate pupils eating

In [7]:
schools_capacities = [size_school] * len(schools)

In [8]:
hexagons = pd.DataFrame()
for i, row in quadrants.iterrows():
    hexs = row.hexagons
    pupils_per_hex = row.pupils_per_hex
    
    for hex_ in hexs:
        if hex_ in hexagons.index:
            hexagons.loc[hex_, "pupils_per_hex"] += pupils_per_hex
        else:
            hexagons.loc[hex_, "pupils_per_hex"] = pupils_per_hex

In [9]:
hexagons_old = hexagons.copy()

In [10]:
for i, school_hexs in enumerate(schools):
    hex_list = []
    for school_hex in school_hexs:
        if school_hex not in hexagons.index:
            continue
        hex_list.append(school_hex)
        
    if len(hex_list) == 0:
        continue
        
    hexagons.loc[hex_list] -= schools_capacities[i] / len(hex_list)
    schools_capacities[i] = 0

# Visualizations

In [11]:
def base_empty_map():
    """Prepares a folium map centered in a central GPS point of Toulouse"""
    m = Map(location = [55.751244, 37.618423],
            zoom_start = 13,
            tiles = "cartodbpositron",
            attr = '''© <a href="http://www.openstreetmap.org/copyright">
                      OpenStreetMap</a>contributors ©
                      <a href="http://cartodb.com/attributions#basemaps">
                      CartoDB</a>'''
            )
    return m

def hexagons_dataframe_to_geojson(df_hex, hex_id_field,
                                  geometry_field, value_field,
                                  file_output = None):

    """Produce the GeoJSON representation containing all geometries in a dataframe
     based on a column in geojson format (geometry_field)"""

    list_features = []

    for i, row in df_hex.iterrows():
        feature = Feature(geometry = row[geometry_field],
                          id = row[hex_id_field],
                          properties = {"value": row[value_field]})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)

    geojson_result = json.dumps(feat_collection)

    # optionally write to file
    if file_output is not None:
        with open(file_output, "w") as f:
            json.dump(feat_collection, f)

    return geojson_result


# --------------------------------------------------------------------


def choropleth_map(df_aggreg, hex_id_field, geometry_field, value_field,
                   layer_name, initial_map = None, kind = "linear",
                   border_color = 'black', fill_opacity = 0.7,
                   with_legend = False):

    """Plots a choropleth map with folium"""

    if initial_map is None:
        initial_map = base_empty_map()

    # the custom colormap depends on the map kind
    if kind == "linear":
        min_value = df_aggreg[value_field].min()
        max_value = df_aggreg[value_field].max()
        m = round((min_value + max_value) / 2, 0)
        custom_cm = cm.LinearColormap(['red', 'white', 'blue'],
                                      vmin = min_value,
                                      vmax = max_value)
    elif kind == "outlier":
        # for outliers, values would be -1,0,1
        custom_cm = cm.LinearColormap(['grey', 'white', 'red'],
                                      vmin=-1, vmax=1)
    elif kind == "filled_nulls":
        min_value = df_aggreg[df_aggreg[value_field] > 0][value_field].min()
        max_value = df_aggreg[df_aggreg[value_field] > 0][value_field].max()
        m = round((min_value + max_value) / 2, 0)
        custom_cm = cm.LinearColormap(['silver', 'blue', 'yellow', 'blue'],
                                      index = [0, min_value, m, max_value],
                                      vmin = min_value,
                                      vmax = max_value)

    # create geojson data from dataframe
    geojson_data = hexagons_dataframe_to_geojson(df_aggreg, hex_id_field,
                                                 geometry_field, value_field)

    # plot on map
    GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': custom_cm(feature['properties']['value']),
            'color': border_color,
            'weight': 1,
            'fillOpacity': fill_opacity
        },
        name = layer_name
    ).add_to(initial_map)

    if with_legend is True:
        custom_cm.add_to(initial_map)

    return initial_map

In [12]:
def hex_to_geojson(hex_id, row):
    coords = h3.h3_to_geo_boundary(hex_id)
    pol_coords = wrapper([list(reversed(x)) for x in coords])
    
    return pol_coords

geojsons = [hex_to_geojson(hex_id, row) for hex_id, row in hexagons.iterrows()]
hexagons["geo"] = geojsons

geojsons = [hex_to_geojson(hex_id, row) for hex_id, row in hexagons_old.iterrows()]
hexagons_old["geo"] = geojsons

## hexagons before pupils eating

Как интерптерировать карту: чем синее цвет, тем больше в данном гексагоне проживает учеников

In [13]:
m_hex = choropleth_map(df_aggreg = hexagons_old.reset_index(),
                       hex_id_field = "index",
                       geometry_field = "geo",
                       value_field = "pupils_per_hex",
                       layer_name = "Choropleth 9",
                       with_legend = True)
m_hex

## after that

Как интерптерировать карту: чем синее цвет, тем больше в данном гексагоне существует потребность в школах. Отрицательные значения означают переизбыток школ

In [14]:
# Для улучшения визуального отображения уберем ячейки с выбросами
hexagons = hexagons[hexagons.pupils_per_hex > -2000]

In [15]:
m_hex = choropleth_map(df_aggreg = hexagons.reset_index(),
                       hex_id_field = "index",
                       geometry_field = "geo",
                       value_field = "pupils_per_hex",
                       layer_name = "Choropleth 9",
                       with_legend = True)
m_hex